# EM (Time/Frequency) 1D Inversion

<img align="right" width="300px" src="./images/inversion_em1d_thumbnail.png">

This application provides an interface to the open-source [SimPEG](https://simpeg.xyz/) package for the inversion of electromagnetic (EM) data using a Laterally Constrained 1D approach. The application supports several known EM systems, for both frequency and time-domain.

New user? Visit the [**Getting Started**](https://geoapps.readthedocs.io/en/latest/content/installation.html) page.

## Application
The following sections provide details on the different parameters controlling the application. Interactive widgets shown below are for demonstration purposes only.

In [1]:
from geoapps.inversion.em1d_inversion_app import InversionApp

app = InversionApp(h5file=r"../../../assets/FlinFlon.geoh5")
app()

## Project Selection

Select and connect to an existing **geoh5** project file containing data. 

In [2]:
app.project_panel

See the [Project Panel](base_application.ipynb#Project-Panel) page for more details.

## Object/Data Selection

List of objects available containing geophysical data to be inverted. Currently supports objects of type `Points`, `Curve`, `Surface` and `Grid2D`. Multiple data channels and/or data groups can be selected. 

In [3]:
app.data_panel

### Geophysical System

List of known EM systems with pre-recorded parameters (frequencies/times, offsets, uncertainties, waveform, etc.). 

**Note:** Users can add custom systems by appending to the configuration file: `geoapps/utils/geophysical_systems.py`

In [4]:
app.system

Dropdown(description='Survey Type: ', index=2, options=('AeroTEM (2007)', 'AeroTEM (2010)', 'DIGHEM', 'GENESIS…

The application will attempt to assign the correct system based on
known data field name (e.g. *CPI* => *DIGHEM*).

## Window Selection

Manual selection of an area of interest and data resolution.

In [5]:
app.window_selection

See the [Map View Selection](view_selection.ipynb#Map-View-Selection) page for more details.

## Input Data

### Channel Options

Set parameters to invividual [Data](#Object/Data-Selection) channels.

In [6]:
app.data_channel_choices.data_channel_options['CPI900']

#### Active

Checked if the channel is to be used in the inversion.

#### Frequency/Time

Specify the frequency (Hz) or time (s) of the given channel.

#### Data channel:

Association between the "data channel" and one of the expected "system channel".

#### Error (%, floor)

Estimated data uncertainties associated with the data channel, expressed as a
percent (decimal) and floor value. 

As an example for magnetic data: 0.1, 1 => $0.1 * |d_{tmi}| + 1\;nT$

#### Offsets (EM systems)

Determine the constant offsets, in meters, between the receiver and the transmitter center location. 

**Note:** Users can edit the default parameters by modifying the configuration file: `geoapps/utils/geophysical_systems.py`

### Line ID

In [7]:
app.lines()

Set the field defining the line `IDs` to be inverted. At least one line must be selected. The selection will be highlighted in the [Window Selection](#Window-Selection) panel.

## Spatial information

### Topography

Set the air/ground interface of the inversion model.

In [8]:
app.topography()

#### Object

Set the topography based on an object stored in the target `geoh5` project. The Z value of the cells/vertices can be assigned based on a chosen `Data` field.

#### Relative to Sensor

Topography is defined by a fixed vertical offset from a selected object position (vertices or centroids), also referred to as a "draped height".

#### Constant

Topography is defined as a flat surface with constant elevation.

### Sensor Location

Defines the sensors position in 3D space. The vertical position can be referenced to:
- **sensor location**: Use the target [object](#Object) vertices or centroids z-locations
- **topo + radar**: Interpolate the topography to set the elevation. Typically used for gridded
    data with constant draped height or for airborne survey with inaccurate GPS
    elevation (see [radar channel](#Radar-channel-(optional))).

In [9]:
app.sensor()

#### Constant offsets (dx, dy, dz)

Sensor position shifted by a constant offset from the vertices of the selected
object. Typically used for towed system where the GPS receiver is on the
aircraft.

- dx: East(+) or West(-) offset.
- dy: North(+) or South(-) offset
- dz: Up(+) or Down(-) offset

#### Radar channel (optional)

Channel defining the ground clearance (radar) of each data point.

## Inversion Options

List of parameters controlling the inversion.

In [10]:
app.inversion_parameters.main

### Output Name

Name given to the inversion group added to the Geoscience ANALYST project.

### Target Misfit

Target data misfit where 1 = number of data

In [11]:
app.inversion_parameters.chi_factor

FloatText(value=1.0, description='Target misfit', style=DescriptionStyle(description_width='initial'))

### Uncertainty mode (optional)

Global change to the data uncertainties based on the forward simulation of the starting (background) model.

In [12]:
app.inversion_parameters.uncert_mode

RadioButtons(index=1, options=('Estimated (%|data| + background)', 'User input (%|data| + floor)'), style=Desc…

**Estimated** Floor uncertainties re-assgined based on the best-fitting halfspace model. Useful when dealing with fast time domain decays due to IP effects.

**User Input** (Default) Keep the same uncertainties as defined in the [Channel Options](#Channel-Options) 

### Starting Model

Initial model used to begin the inversion.


In [13]:
app.inversion_parameters.starting_model.main

#### Model

Model object and values selected from any `Points`, `Curve`, `Surface`, `BlockModel` or `Octree` object.
Values are interpolated onto the inversion mesh using a nearest neighbor approach.

#### Constant

Constant background half-space value.

### Susceptibility model 

**(Frequency EM only)**

Susceptibility values used in the forward calculations only.

In [14]:
app.inversion_parameters.susceptibility_model.main

#### None

No susceptibility model used.

#### Model

Susceptibitlity model selected from any `Points`, `Curve`, `Surface`, `BlockModel` or `Octree` object.
Values are interpolated onto the inversion mesh using a nearest neighbor approach.

#### Constant

Constant susceptibility half-space value.

### Regularization

See the [Regularization Panel](#Regularization-Panel) section.

### Upper-Lower Bounds

Upper and lower bound constraints applied on physical property model.

In [15]:
app.inversion_parameters.bound_panel

If left empty, the inversion uses [$-\infty$, $\infty$] physical property bounds.

### Ignore Values

#### Data

Flag value ignored by the inversion by assigning $\infty$ uncertainties on the data points.

In [16]:
app.inversion_parameters.ignore_values

Text(value='<0', description='Data (i.e. <0 = no negatives)', style=DescriptionStyle(description_width='initia…

### Optimization

Parameters controlling various aspects of the projected Gauss-Newton inversion algorithm.

In [17]:
app.inversion_parameters.optimization

#### Max beta Iterations

Maximum number of $\beta$-iterations allowed.
Note that when applying sparse norms, the inversion may require $>20$ iterations to converge.


#### Target misfit

Target data misfit where $\chi=1$ corresponds to $\phi_d=N$ (number of data). (See [Data Misfit and Uncertainties](https://giftoolscookbook.readthedocs.io/en/latest/content/fundamentals/Uncertainties.html#data-misfit-and-uncertainties))


#### Starting trade-off ($\beta$)

**ratio**: Factor multiplying the initial $\beta$ defined by the ratio between the initial misfit and regularization:

\begin{equation}
\beta_0 = \gamma * \frac{\phi_d}{ \phi_m}
\end{equation}

**value**: Fixed $\beta$ value specified by the user.

#### Max CG Iterations

Maximum number of Conjugate Gradient (CG) iterations per Gauss-Newton solve.


#### CG Tolerance

Threshold on the minimum Conjugate Gradient (CG) step to end the Gauss-Newton solve.


#### Max CPUs

Maximum number of threads used for the parallelization. Defaults to half the system thread count.


#### Max RAM (Gb)

Approximate memory (RAM) used during the inversion. The value is used to calculate the chunk size for the storage of the sensitivity matrix on a solid-state drive.
(See [dask.array.to_zarr](https://docs.dask.org/en/latest/array-api.html#dask.array.to_zarr))


## Regularization Panel

Parameters controlling the regularization function.

### Reference model

Reference model values used to constrain the inversion.

In [18]:
app.inversion_parameters.reference_model.main

#### None

No reference model used. This is equivalent as setting the [Alphas](#Alpha-Parameters) `s` parameter to zero.

#### Model

Reference model selected from any `Points`, `Curve`, `Surface`, `BlockModel` or `Octree` object.
Values are interpolated onto the inversion mesh using a nearest neighbor approach.

#### Constant

Constant reference half-space value.

#### Best-fitting halfspace

Run a preliminary inversion to determine a best-fitting halfspace
(single conductivity value) at each data station.

### Alpha Parameters

Scaling between the components of the regularization function.

In [19]:
app.inversion_parameters.alphas

Text(value='1.0, 1.0, 1.0, 1.0', description='Scaling alpha_(s, x, y, z)', style=DescriptionStyle(description_…

See the [SimPEG.API](https://docs.simpeg.xyz/content/api_core/api_Regularization.html#SimPEG.regularization.Simple) for technical details.

### $l_p$-norms

Approximated norms applied to the components of the regularization.

In [20]:
app.inversion_parameters.norms


Text(value='2, 2, 2, 2', continuous_update=False, description='Norms p_(s, x, y, z)', style=DescriptionStyle(d…

See notes on [Sparse and Blocky Norms](https://giftoolscookbook.readthedocs.io/en/latest/content/fundamentals/Norms.html#sparse-and-blocky-norms) for technical details.

### Mesh

Define the 1D layer mesh. The same mesh is copied below each transmitter/receiver pairs. For the laterally constrained 1D inversion, neighboring layer meshes are connected by the regularization function.

In [21]:
app.mesh_1D.main

#### Smallest cell (m)

Smallest layer at the top of the 1D mesh.


#### Expansion factor

Rate of cell expansion from the smallest cell size.


#### Number of cells

Total number of layers below each station.


#### Max depth

Returns the full depth of the 1D mesh given the parameters selected above.

## Output panel

Setup the inversion and launch the process.

### Output Name

In [22]:
app.ga_group_name

Text(value='EM1DInversion_', continuous_update=False, description='Group:', style=DescriptionStyle(description…

Name given to the inversion group added to the Geoscience ANALYST project.

### Working directory

In [23]:
app.export_directory

FileChooser(path='C:\Users\dominiquef\Documents\GIT\mira\geoapps\docs\content\applications', filename='', titl…

Specify the working directory where the inversion occurs. 

**It is highly recommended to chose a location on a solid-state drive (SSD) to fully take advantage of the `Dask` parallelization.**

### Write input

In [24]:
app.write

Button(button_style='warning', description='Write input', icon='check', style=ButtonStyle())


Click to write the input parameters to a `*.json` file and a workspace (.geoh5) with all required entities. 

### Run Inversion: 

In [25]:
app.trigger

Button(button_style='danger', description='Compute', icon='check', style=ButtonStyle(), tooltip='Run computati…

Launch the inversion routine. Results will be written directly to the target `geoh5` project.

**Write input**: Click to write the input parameters to a `*.json` file

**Run SimPEG**: Launch the inversion routine. Results will be written directly to the target `geoh5` project.

### Command line execution

Alternatively, the inversion can be launched on a different computer from command line:

`activate geoapps`

`python -m geoapps.drivers.em1d_inversion [Name].json`

This assumes that the geoapps have been installed and that the reference `*.json` and `*.geoh5` project are accessible.

In [26]:
import matplotlib.pyplot as plt
app.figure.savefig("images/inversion_thumbnail.png", dpi=300, bbox_inches="tight")